In [54]:
import sys
import json
import os
from IPython.nbformat import read, write, NO_CONVERT

In [57]:
def _cells(nb):
    """Yield all cells in an nbformat-insensitive manner"""
    if nb.nbformat < 4:
        for ws in nb.worksheets:
            for cell in ws.cells:
                yield cell
    else:
        for cell in nb.cells:
            yield cell
            
def strip_output(nb):
    """
    Strip the outputs, execution count/prompt number and miscellaneous
    metadata from a notebook object, unless specified to keep either the outputs
    or counts.
    """

    nb.metadata.pop('signature', None)
    nb.metadata.pop('widgets', None)

    for cell in _cells(nb):

        keep_output_this_cell = False
        keep_count = False

        # Remove the outputs, unless directed otherwise
        if 'outputs' in cell:

            # Default behavior strips outputs. With all outputs stripped,
            # there are no counts to keep and keep_count is ignored.
            if not keep_output_this_cell:
                cell['outputs'] = []

            # If keep_output_this_cell, but not keep_count, strip the counts
            # from the output.
            if keep_output_this_cell and not keep_count:
                for output in cell['outputs']:
                    if 'execution_count' in output:
                        output['execution_count'] = None

            # If keep_output_this_cell and keep_count, do nothing.

        # Remove the prompt_number/execution_count, unless directed otherwise
        if 'prompt_number' in cell and not keep_count:
            cell['prompt_number'] = None
        if 'execution_count' in cell and not keep_count:
            cell['execution_count'] = None

        # Always remove this metadata
        for output_style in ['collapsed', 'scrolled']:
            if output_style in cell.metadata:
                cell.metadata[output_style] = False
        if 'metadata' in cell:
            for field in ['collapsed', 'scrolled', 'ExecuteTime']:
                cell.metadata.pop(field, None)
    return nb

In [58]:
filename = 'Edgar.ipynb'
with io.open(filename, 'r', encoding='utf8') as f:
    nb = read(f, as_version=NO_CONVERT)
nb = strip_output(nb)    

In [59]:
filename = 'Edgar_removed.ipynb'
with io.open(filename, 'w', encoding='utf8') as f:
    write(nb, f)